<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-07 13:32:07
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.58 C
-------------------
Today PnL: -6.40 K (-0.04%)
Current PnL: -25.30 L (-16.13%)
CY Booked + Current PnL: -11.01 L (-7.02%)
-------------------
Total profit:  2.21 L
Total loss:  -27.51 L
-------------------
Total Booked + Current PnL: 15.97 L (12.3%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (9.05%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.52 C
Est FTT PnL: 94.31 L (59.75%)
Deployed:  1.30 C
Current:  1.58 C
CAGR/XIRR %: 10.88%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,NESTLEIND,1377.00,-0.37,15.93,4.71,21.40,14493.0,42284.0,307710.0,7.60,68.0,X-LC,5.24,14.0,2.92,1.95,23.27,XY25,NTT,FMCG
49,MASFIN,398.61,-0.10,3.41,18.02,22.05,18259.0,3345.0,101325.0,-11.02,64.0,H-SC,6.96,168.0,0.18,0.64,47.28,XR,ATH,FINANCE
17,COALINDIA,484.83,0.26,5.29,13.24,19.23,20974.0,7956.0,158416.0,24.27,82.0,L-LC,6.27,182.0,0.38,1.00,27.69,XY25,ATH,MINING
80,TTKPRESTIG,770.00,-1.53,-20.89,26.52,0.09,21144.0,-21050.0,79727.0,85.06,37.0,M-SC,3.21,253.0,-1.00,0.51,4.74,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,1.80,-13.91,16.18,0.02,24315.0,-24283.0,150276.0,129.35,56.0,M-SC,7.50,240.0,-1.00,0.95,30.71,OX40N,NTT,PAINTS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TATAELXSI,9161.00,7.95,-16.84,58.77,32.03,64430.0,-22205.0,109630.0,-9.99,71.0,H-SC,7.02,157.0,-0.34,0.69,22.79,OX40N,NTT,IT
23,DMART,5391.45,4.92,-7.60,40.26,29.60,92853.0,-18966.0,230634.0,-21.24,52.0,X-LC,9.64,10.0,-0.20,1.46,12.92,X40N,ATH,FMCG
13,BSOFT,831.70,4.30,-20.34,89.28,50.79,100036.0,-28606.0,112047.0,2.05,57.0,H-SC,10.61,171.0,-0.29,0.71,30.46,XR,ATH,IT
44,JCHAC,2282.00,2.71,-35.32,54.60,-0.01,45133.0,-45143.0,82662.0,16813.04,47.0,M-SC,14.15,235.0,-1.00,0.52,5.80,OX40N,BTT,AC
54,QUESS,424.00,2.66,-26.33,93.88,42.84,44962.0,-17113.0,47893.0,-51.52,57.0,X-SC,29.20,83.0,-0.38,0.30,8.35,XY24,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJOOENG,143.10,-5.86,-28.63,95.63,39.61,69953.0,-29350.0,73150.0,-51.80,58.0,H-SC,31.71,136.0,-0.42,0.46,18.40,AR,ATH,MISC
78,TMPV,600.00,-2.33,-26.15,66.53,22.98,110985.0,-59079.0,166819.0,-23.08,47.0,X-LC,4.75,2.0,-0.53,1.06,4.92,XY24,NTT,AUTO
40,INDUSINDBK,1800.00,-2.27,-34.03,101.45,32.89,51668.0,-26276.0,50930.0,-728.69,62.0,L-MC,7.43,259.0,-0.51,0.32,40.26,XR,NTT,BANKS
35,ICICIGI,2252.93,-1.91,4.44,14.27,19.34,29823.0,8881.0,208990.0,-17.67,51.0,X-MC,3.81,71.0,0.30,1.32,21.08,X40,ATH,INSURANCE
79,TRIDENT,48.00,-1.80,-29.99,83.56,28.51,53971.0,-27664.0,64590.0,-17.17,32.0,M-SC,7.09,226.0,-0.51,0.41,10.52,XR,NTT,TEXTILES


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.08,-0.89,109.78,107.91,160639.0,-1316.0,146328.0,-19.34,56.0,M-SC,10.16,220.0,-0.01,0.93,9.1,OX40N,ATH,CABLES


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.08,-0.89,109.78,107.91,160639.0,-1316.0,146328.0,-19.34,56.0,M-SC,10.16,220.0,-0.01,0.93,9.10,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,1.80,-13.91,16.18,0.02,24315.0,-24283.0,150276.0,129.35,56.0,M-SC,7.50,240.0,-1.00,0.95,30.71,OX40N,NTT,PAINTS
75,TATAELXSI,9161.00,7.95,-16.84,58.77,32.03,64430.0,-22205.0,109630.0,-9.99,71.0,H-SC,7.02,157.0,-0.34,0.69,22.79,OX40N,NTT,IT
47,KANSAINER,340.00,-0.52,-20.23,42.25,13.47,90884.0,-54558.0,215109.0,-67.15,57.0,H-SC,5.00,159.0,-0.60,1.36,9.38,XY24,NTT,PAINTS
80,TTKPRESTIG,770.00,-1.53,-20.89,26.52,0.09,21144.0,-21050.0,79727.0,85.06,37.0,M-SC,3.21,253.0,-1.00,0.51,4.74,OX40N,NTT,DURABLES


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.00,-0.10,3.53,16.62,20.74,34960.0,7178.0,210348.0,35.83,73.0,M-MC,6.33,189.0,0.21,1.33,14.52,XY25,NTT,FINANCE
17,COALINDIA,484.83,0.26,5.29,13.24,19.23,20974.0,7956.0,158416.0,24.27,82.0,L-LC,6.27,182.0,0.38,1.00,27.69,XY25,ATH,MINING
87,WIPRO,420.00,1.30,10.35,56.10,72.27,104901.0,17542.0,186990.0,-5.17,61.0,M-LC,6.04,101.0,0.17,1.18,17.17,XR,NTT,IT
49,MASFIN,398.61,-0.10,3.41,18.02,22.05,18259.0,3345.0,101325.0,-11.02,64.0,H-SC,6.96,168.0,0.18,0.64,47.28,XR,ATH,FINANCE
37,IEX,219.00,2.11,3.02,44.27,48.63,91193.0,6039.0,205993.0,-31.50,69.0,H-SC,13.92,137.0,0.07,1.31,14.99,XR,NTT,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,MASFIN,398.61,-0.10,3.41,18.02,22.05,18259.0,3345.0,101325.0,-11.02,64.0,H-SC,6.96,168.0,0.18,0.64,47.28,XR,ATH,FINANCE
87,WIPRO,420.00,1.30,10.35,56.10,72.27,104901.0,17542.0,186990.0,-5.17,61.0,M-LC,6.04,101.0,0.17,1.18,17.17,XR,NTT,IT
37,IEX,219.00,2.11,3.02,44.27,48.63,91193.0,6039.0,205993.0,-31.50,69.0,H-SC,13.92,137.0,0.07,1.31,14.99,XR,NTT,MISC
25,FINCABLES,1641.55,0.08,-0.89,109.78,107.91,160639.0,-1316.0,146328.0,-19.34,56.0,M-SC,10.16,220.0,-0.01,0.93,9.10,OX40N,ATH,CABLES
38,INDIAMART,4810.62,0.24,-6.63,121.41,106.72,139807.0,-8183.0,115153.0,-53.68,38.0,H-SC,7.01,138.0,-0.06,0.73,15.19,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-0.50,-16.48,32.65,10.78,65529.0,-39616.0,200702.0,-50.14,13.0,X-LC,5.02,1.0,-0.60,1.27,0.00,X40,NTT,FMCG
22,DIXON,18931.72,-0.49,-19.09,62.24,31.26,116205.0,-44064.0,186704.0,-62.24,27.0,X-MC,18.20,56.0,-0.38,1.18,0.00,X40N,ATH,IT
7,AWL,485.00,-0.24,-25.38,110.65,57.19,269282.0,-82763.0,243364.0,-64.78,29.0,X-MC,8.42,58.0,-0.31,1.54,0.00,XY24,NTT,FMCG
52,PAGEIND,51605.07,0.08,-10.43,45.82,30.62,97294.0,-24714.0,212340.0,-36.35,33.0,X-MC,16.01,57.0,-0.25,1.35,0.08,X40,ATH,APPARELS
11,BATAINDIA,2096.00,-1.07,-39.64,123.98,35.20,96297.0,-50999.0,77671.0,1.81,36.0,X-SC,17.26,93.0,-0.53,0.49,0.00,X40,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,UNITDSPR,1644.00,-0.35,0.77,19.84,20.77,46812.0,1806.0,235950.0,-8.06,38.0,X-MC,0.97,70.0,0.04,1.49,7.74,X40N,NTT,BREWERIES
68,SIEMENS,4671.50,-0.52,-16.39,50.12,25.51,77982.0,-30505.0,155590.0,-4.56,48.0,H-LC,1.43,50.0,-0.39,0.99,14.66,AR,ATH,ELECTRICAL
20,DABUR,735.00,0.15,1.89,41.03,43.70,103707.0,4690.0,252758.0,-6.50,66.0,X-MC,2.01,73.0,0.05,1.60,17.23,XY24,BTT,FMCG
84,VBL,671.64,0.67,1.44,33.74,35.67,108104.0,4562.0,320404.0,-10.61,68.0,X-LC,2.13,5.0,0.04,2.03,15.50,X40N,ATH,FMCG
45,JIOFIN,387.00,1.49,-2.54,27.72,24.48,72233.0,-6785.0,260580.0,-4.17,56.0,X-LC,2.28,37.0,-0.09,1.65,51.04,XY24,BTT,FINANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,AWL,485.00,-0.24,-25.38,110.65,57.19,269282.0,-82763.0,243364.0,-64.78,29.0,X-MC,8.42,58.0,-0.31,1.54,0.00,XY24,NTT,FMCG
11,BATAINDIA,2096.00,-1.07,-39.64,123.98,35.20,96297.0,-50999.0,77671.0,1.81,36.0,X-SC,17.26,93.0,-0.53,0.49,0.00,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-0.49,-19.09,62.24,31.26,116205.0,-44064.0,186704.0,-62.24,27.0,X-MC,18.20,56.0,-0.38,1.18,0.00,X40N,ATH,IT
43,ITC,452.00,-0.50,-16.48,32.65,10.78,65529.0,-39616.0,200702.0,-50.14,13.0,X-LC,5.02,1.0,-0.60,1.27,0.00,X40,NTT,FMCG
52,PAGEIND,51605.07,0.08,-10.43,45.82,30.62,97294.0,-24714.0,212340.0,-36.35,33.0,X-MC,16.01,57.0,-0.25,1.35,0.08,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,2.66,-26.33,93.88,42.84,44962.0,-17113.0,47893.0,-51.52,57.0,X-SC,29.20,83.0,-0.38,0.30,8.35,XY24,NTT,MISC
58,RELAXO,1176.00,-0.93,-48.12,193.56,52.31,145776.0,-69847.0,75313.0,-44.73,42.0,X-SC,7.61,92.0,-0.48,0.48,0.81,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.07,-39.64,123.98,35.20,96297.0,-50999.0,77671.0,1.81,36.0,X-SC,17.26,93.0,-0.53,0.49,0.00,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,2.38,3.34,20.06,24.08,26318.0,4246.0,131196.0,-5.01,60.0,X-SC,6.71,91.0,0.16,0.83,23.62,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.41,-16.41,71.46,43.32,97286.0,-26732.0,136140.0,-28.18,51.0,X-SC,7.78,90.0,-0.27,0.86,4.53,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-0.50,-16.48,32.65,10.78,65529.0,-39616.0,200702.0,-50.14,13.0,X-LC,5.02,1.0,-0.60,1.27,0.00,X40,NTT,FMCG
78,TMPV,600.00,-2.33,-26.15,66.53,22.98,110985.0,-59079.0,166819.0,-23.08,47.0,X-LC,4.75,2.0,-0.53,1.06,4.92,XY24,NTT,AUTO
76,TCS,4389.96,0.94,-9.63,33.58,20.71,114775.0,-36441.0,341796.0,-22.01,59.0,X-LC,2.67,3.0,-0.32,2.17,14.21,X40,ATH,IT
84,VBL,671.64,0.67,1.44,33.74,35.67,108104.0,4562.0,320404.0,-10.61,68.0,X-LC,2.13,5.0,0.04,2.03,15.50,X40N,ATH,FMCG
41,INFY,1972.00,1.35,11.24,20.69,34.24,72010.0,35154.0,348042.0,-13.50,55.0,X-LC,7.55,6.0,0.49,2.21,20.45,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-2.27,-34.03,101.45,32.89,51668.0,-26276.0,50930.0,-728.69,62.0,L-MC,7.43,259.0,-0.51,0.32,40.26,XR,NTT,BANKS
49,MASFIN,398.61,-0.10,3.41,18.02,22.05,18259.0,3345.0,101325.0,-11.02,64.0,H-SC,6.96,168.0,0.18,0.64,47.28,XR,ATH,FINANCE
13,BSOFT,831.70,4.30,-20.34,89.28,50.79,100036.0,-28606.0,112047.0,2.05,57.0,H-SC,10.61,171.0,-0.29,0.71,30.46,XR,ATH,IT
72,SURYODAY,216.00,-1.33,-18.91,51.72,23.03,75106.0,-33854.0,145217.0,57.89,54.0,H-SC,5.19,165.0,-0.45,0.92,44.08,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,1.80,-13.91,16.18,0.02,24315.0,-24283.0,150276.0,129.35,56.0,M-SC,7.50,240.0,-1.00,0.95,30.71,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,-1.20,-27.02,54.71,12.90,87237.0,-59043.0,159453.0,134.38,70.0,H-SC,10.60,173.0,-0.68,1.01,32.58,OX40N,NTT,FINANCE
75,TATAELXSI,9161.00,7.95,-16.84,58.77,32.03,64430.0,-22205.0,109630.0,-9.99,71.0,H-SC,7.02,157.0,-0.34,0.69,22.79,OX40N,NTT,IT
85,VOLTAS,1856.00,1.37,17.86,23.19,45.20,52407.0,34248.0,225990.0,6.38,73.0,X-MC,4.14,80.0,0.65,1.43,25.87,XY25,NTT,AC
17,COALINDIA,484.83,0.26,5.29,13.24,19.23,20974.0,7956.0,158416.0,24.27,82.0,L-LC,6.27,182.0,0.38,1.00,27.69,XY25,ATH,MINING
57,RECLTD,446.00,-0.10,3.53,16.62,20.74,34960.0,7178.0,210348.0,35.83,73.0,M-MC,6.33,189.0,0.21,1.33,14.52,XY25,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,21.84
1,25,44.83
2,50,75.51


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

,CurrAlloc%
Conviction,
SC,39.16
LC,32.72
MC,28.09


In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

,CurrAlloc%
Criteria,
XY24,24.14
X40,21.75
XY25,18.38
X40N,11.65
XR,8.33
AR,7.88
OX40N,6.97
SR,0.87


In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

,CurrAlloc%
Conviction,
X-LC,29.55
X-MC,22.17
H-SC,20.98
M-SC,10.28
X-SC,7.17
H-MC,4.27
M-MC,1.33
M-LC,1.18
L-LC,1.00


In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.50,-7.55,42.84
IT,11.70,-15.76,74.15
FINANCE,10.20,-10.65,57.09
REFINERIES,9.84,-3.68,29.92
MISC,6.56,-25.38,76.67
ELECTRICAL,5.42,-9.80,50.02
PAINTS,5.23,-11.86,28.47
INSURANCE,4.27,-0.52,34.91
PHARMA,3.62,-2.75,35.09


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3158339.0,21
AR,1327918.0,10
XR,1177015.0,12
X40,1087960.0,15
X40N,901107.0,9
XY25,804227.0,10
OX40N,691199.0,10
SR,283629.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3481032.0,24
X-MC,1532564.0,16
M-SC,1435498.0,15
X-LC,1325228.0,16
X-SC,785146.0,8
H-MC,405013.0,3
L-SC,176428.0,2
M-LC,104901.0,1
H-LC,77982.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1233491.0      6
           AR         911906.0      5
M-SC       XY24       824397.0      6
H-SC       XR         752135.0      7
X-LC       XY25       530310.0      6
X-MC       X40        486883.0      7
           XY24       435987.0      3
X-LC       X40        410743.0      6
X-MC       X40N       391711.0      4
X-SC       X40N       308439.0      3
M-SC       OX40N      300765.0      5
H-SC       OX40N      299871.0      4
X-SC       XY24       286373.0      3
H-SC       SR         283629.0      2
X-MC       XY25       217983.0      2
H-MC       AR         210140.0      2
X-LC       X40N       200957.0      2
H-MC       XY24       194873.0      1
X-SC       X40        190334.0      2
X-LC       XY24       183218.0      2
M-SC       XR         182446.0      2
           AR         127890.0      2
M-LC       XR         104901.0      1
L-SC       OX40N       90563.0      1
           XR          85865.0      1
H-LC       AR          77982.0      1
L-MC       XR          51668.0      1
M-MC       XY25        34960.0      1
L-LC       XY25        20974.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
